In [1]:
cd /home/daliu/Src/spykshrk_realtime/

/home/daliu/Src/spykshrk_realtime


In [61]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import patches
import json
import os
import scipy.signal

from spykshrk.realtime.simulator import nspike_data

from spykshrk.franklab.pp_decoder.util import gaussian, normal2D, apply_no_anim_boundary, simplify_pos_pandas
from spykshrk.franklab.pp_decoder.pp_clusterless import OfflinePPDecoder
from spykshrk.franklab.pp_decoder.data_containers import EncodeSettings, DecodeSettings, SpikeObservation, \
                                                         LinearPosition, StimLockout, Posteriors, \
                                                         FlatLinearPosition, SpikeWaves, SpikeFeatures

from spykshrk.franklab.pp_decoder.visualization import DecodeVisualizer
from spykshrk.franklab.pp_decoder.decode_error import LinearDecodeError

        
%load_ext Cython

%matplotlib inline

#pd.set_option('float_format', '{:,.2f}'.format)
pd.set_option('display.precision', 4)
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 15)
#pd.set_option('display.width', 80)

idx = pd.IndexSlice

matplotlib.rcParams.update({'font.size': 28})



The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [66]:
%%time
# Load merged rec HDF store based on config

#config_file = '/opt/data36/daliu/realtime/spykshrk/ripple_dec/bond.config.json'
#config_file = '/opt/data36/daliu/realtime/spykshrk/dec_60uv/bond.config.json'
config_file = '/home/daliu/Src/spykshrk_realtime/config/bond_single.json'
config = json.load(open(config_file, 'r'))

day = config['simulator']['nspike_animal_info']['days'][0]
epoch = config['simulator']['nspike_animal_info']['epochs'][0]
time_bin_size = config['pp_decoder']['bin_size']

# Extract just encode and decode settings from config
encode_settings = EncodeSettings(config)
decode_settings = DecodeSettings(config)

# Grab animal linearized real position
nspike_anim = nspike_data.AnimalInfo(**config['simulator']['nspike_animal_info'])
pos = nspike_data.PosMatDataStream(nspike_anim)
pos_data = pos.data

spk = nspike_data.SpkDataStream(nspike_anim)
spk_data = SpikeWaves(spk.data)

# Encapsulate linear position
lin_obj = LinearPosition.from_nspike_posmat(pos_data, encode_settings)

CPU times: user 6.86 s, sys: 660 ms, total: 7.52 s
Wall time: 7.52 s


In [67]:
spk_amp = spk_data.max(axis=1)
spk_amp = spk_amp.to_frame().pivot_table(index=['day','epoch','tet','timestamp','time'], 
                                         columns='channel', values=0)
spk_amp = SpikeFeatures(spk_amp)

In [64]:
spk_amp_thresh = spk_amp.get_above_threshold(60)

spk_amp_thresh.groupby(tet)

[channel                            0   1   2    3
 day epoch tet timestamp time                     
 4   1     5   73831218  2,461.0   24  23  54   21
               73831404  2,461.0   43  59  48   56
               73831440  2,461.0   59  46  65   56
               73831794  2,461.1  339  43  59  128
               73832679  2,461.1   26  85  29   36
 ...                              ...  ..  ..  ...
           29  102148872 3,405.0   87  37  43   73
               102148959 3,405.0   25  35  33   60
               102149112 3,405.0   84  40  58   78
               102149151 3,405.0   33  57  23   31
               102149298 3,405.0   54  46  26   35
 
 [409340 rows x 4 columns],
 <SpikeFeatures: 1df875c0-06fe-433a-92ba-8fe7e72bcb72, shape: ((409340, 4))>]

In [65]:
grp = spk_amp_thresh.groupby('tet')
for grp_ii, grp_entry in grp:
    print(grp_ii, ': ', len(grp_entry))

5 :  18105
11 :  53863
12 :  13821
13 :  44689
14 :  60814
17 :  24183
19 :  45430
29 :  42206
